In [2]:
import importlib
import numpy as np

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

# Classification Results

In [3]:
league_standing_pytable = MyPyTable()
league_standing_pytable.load_from_file("lib/nhl_leaguestandings.csv")

In [ ]:
# descision tree
y_col_name = "PLAYOFFS"

y = league_standing_pytable.get_column(y_col_name)
league_standing_pytable.drop_col(y_col_name)
X = league_standing_pytable.data


X_train_folds, X_test_folds  = myevaluation.stratified_kfold_cross_validation(X, y, 10)

descision_tree = MyDecisionTreeClassifier()
descision_tree.fit()